在本案例研究中，我们使用了匈牙利水痘病例数据集。我们将训练一个回归器，使用循环图卷积网络来预测各县每周报告的病例。

首先，我们将加载数据集并创建适当的时空分割。

In [1]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

在接下来的步骤中，我们将定义用于解决监督任务的递归图神经网络架构。构造函数定义了一个DCRNN层和一个前馈层。
重要的是要注意，最终的非线性没有集成到递归图卷积操作中。这个设计原则是一致使用的，它取自PyTorch Geometric。因此，我们手动定义了递归层和线性层之间的ReLU非线性。当我们解决具有零均值目标的回归问题时，最后的线性层不会跟着非线性。

<html>
<img src="./DCRNN.jpg" width="60%">

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

让我们定义一个模型(我们有4个节点特征)，并在训练集(前20%的时间快照)上训练它200个epoch。当每个时间快照的损失累积时，我们进行反向传播。我们将使用学习率为0.01的Adam优化器。tqdm函数用于度量每个训练周期的运行时间需求。

In [3]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 200/200 [00:16<00:00, 11.98it/s]


让我们定义一个模型(我们有4个节点特征)并使用holdout对其进行训练我们将评估经过训练的递归图卷积网络的性能，并**计算所有空间单元和时间周期的均方误差**。

In [4]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 1.0308
